In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Reshape, LSTM
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

In [10]:
data_directory = './data/Train-Test Set 2'

transform = transforms.Compose([
    # transforms.Resize((640, 480)),
    # transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

custom_dataset = ImageFolder(root=data_directory, transform=transform)

In [11]:
torch.manual_seed(42)

In [12]:
batch_size = 16
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [16]:
model2 = Sequential()

model2.add(Conv2D(128, (3, 3), activation='relu', input_shape=(640, 480, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Reshape((4524, 128)))
model2.add(LSTM(256))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
# model2.add(Dropout(.1))
model2.add(Dense(4, activation='softmax'))

In [17]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 638, 478, 128)     3584      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 319, 239, 128)     0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 317, 237, 128)     147584    
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 158, 118, 128)     0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 156, 116, 128)     147584    
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 78, 58, 128)      

In [19]:
train_datagen = ImageDataGenerator(
    # rescale=1./255,
)
test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    './data/Train-Test Set/Training',
    target_size=(640, 480),
    batch_size=8,
    class_mode='sparse'
)

validation_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set/Validation',
    target_size=(640, 480),
    batch_size=8,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set/Testing',
    target_size=(640, 480),
    batch_size=8,
    class_mode='sparse'
)

Found 169 images belonging to 4 classes.
Found 114 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [20]:
model2.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/10


C:\Users\user\AppData\Local\Temp\ipykernel_8264\1707472198.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(


22/22 [==============================] - 207s 9s/step - loss: 1.4068 - accuracy: 0.3254 - val_loss: 1.3293 - val_accuracy: 0.3947
Epoch 2/10
22/22 [==============================] - 192s 9s/step - loss: 1.3444 - accuracy: 0.3728 - val_loss: 1.3623 - val_accuracy: 0.3947
Epoch 3/10
 1/22 [>.............................] - ETA: 3:08 - loss: 1.2628 - accuracy: 0.3750

KeyboardInterrupt: 

In [16]:
model2.save('cnn_lstm_3947.h5')

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
from sklearn.metrics import confusion_matrix
import itertools

predictions = model2.predict_generator(test_generator)
y_pred = np.round(predictions).astype(int).flatten()  # Convert probabilities to class labels
# print(y_true, y_pred)
y_true = test_generator.classes
print(y_true)
# cm = confusion_matrix(y_true, y_pred)

# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

# plot_confusion_matrix(cm, classes=validation_generator.class_indices.keys())

# plt.show()

C:\Users\user\AppData\Local\Temp\ipykernel_10156\1749941455.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model2.predict_generator(test_generator)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3]
